Here's a summary of the 6 findings:

1. New York has the highest Medicare spend per enrollee at $14,140
2. Hawaii has the lowest at $8,221 — a $5,919 gap vs New York
3. Children under 18 on Medicare are the highest cost age group at $60,014 per enrollee
4. American Indian/Alaska Native beneficiaries have the highest racial spend at $14,896
5. COVID visibly depressed spending in 2020 across all service types except Hospice
6. Dual Medicaid enrollees cost 2x more than non-dual enrollees
7. Cost burden by state -Iowa highest burden at 21.3%
8. Racial disparity index - AIAN 23.9% above average, API 27.2% below

In [24]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data/medicare_eda.db')

# Load the three cleaned CSVs into the database as tables
df_states = pd.read_csv('C:/Users/16503/Downloads/medicare-eda-project/output/ab5_states_2023.csv')
df_demo = pd.read_csv('C:/Users/16503/Downloads/medicare-eda-project/output/ab4_demographics_2023.csv')
df_trends = pd.read_csv('C:/Users/16503/Downloads/medicare-eda-project/output/ab1_trends_2018_2023.csv')

df_states.to_sql('states', conn, if_exists='replace', index=False)
df_demo.to_sql('demographics', conn, if_exists='replace', index=False)
df_trends.to_sql('trends', conn, if_exists='replace', index=False)

print("Tables loaded successfully!")

# Verify
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print("Tables in database:", cursor.fetchall())

Tables loaded successfully!
Tables in database: [('states',), ('demographics',), ('trends',)]


In [4]:
# Helper function to run SQL and display results cleanly
def run_query(sql):
    return pd.read_sql_query(sql, conn)

# Query 1: Top 10 states by payments per enrollee
q1 = """
SELECT 
    state,
    total_enrollees,
    ROUND(total_program_payments / 1000000000, 2) AS total_payments_billions,
    ROUND(payments_per_enrollee, 0) AS payments_per_enrollee
FROM states
ORDER BY payments_per_enrollee DESC
LIMIT 10
"""

run_query(q1)

,State,Total_enrollees,total_payments_billions,payments_per_enrollee
0,New York,1908935,26.0,14140.0
1,California,3401066,47.0,14085.0
2,Florida,2281358,30.0,13344.0
3,New Jersey,1055481,13.0,13171.0
4,Connecticut,331320,4.0,12834.0
5,Texas,2229266,28.0,12775.0
6,Maryland,876946,11.0,12675.0
7,Louisiana,427858,5.0,12592.0
8,Nevada,299208,3.0,12519.0
9,Massachusetts,947643,11.0,12474.0


New York has the highest Medicare spend per enrollee at $14,140 even higher than California despite California having nearly twice as many enrollees. This suggests New York's costs are driven by intensity of care rather than just population size. Connecticut showing up at #5 is also notable for a small state.

In [6]:
# Query 2: Bottom 10 states by payments per enrollee
q2 = """
SELECT 
    state,
    total_enrollees,
    ROUND(total_program_payments / 1000000000, 2) AS total_payments_billions,
    ROUND(payments_per_enrollee, 0) AS payments_per_enrollee
FROM states
ORDER BY payments_per_enrollee ASC
LIMIT 10
"""

run_query(q2)

,State,Total_enrollees,total_payments_billions,payments_per_enrollee
0,Hawaii,141898,1.0,8221.0
1,Montana,185502,1.0,9170.0
2,New Mexico,234663,2.0,9481.0
3,Washington,815011,7.0,9485.0
4,Oregon,434831,4.0,9521.0
5,Maine,166638,1.0,9646.0
6,Idaho,204709,1.0,9761.0
7,New Hampshire,225191,2.0,9992.0
8,Virginia,1053211,10.0,10209.0
9,Wyoming,107258,1.0,10341.0


Hawaii is the lowest cost state at just $8,221 per enrollee nearly half of New York's $14,140. The spread between highest and lowest is over $5,900 per person which is a compelling finding for your project.
The pattern is also interesting. The lowest cost states are mostly Western states (Hawaii, Montana, Oregon, Washington, Idaho) while the highest cost states are mostly Northeastern. 

In [8]:
# Query 3: Payments per enrollee by age group
q3 = """
SELECT 
    demographic_group,
    total_enrollees,
    ROUND(payments_per_enrollee, 0) AS payments_per_enrollee,
    ROUND(total_cost_sharing, 0) AS total_cost_sharing
FROM demographics
WHERE demographic_group IN (
    'Under 18 Years', '18-24 Years', '25-34 Years', 
    '35-44 Years', '45-54 Years', '55-64 Years',
    '65-74 Years', '75-84 Years', '85-94 Years', 
    '95 Years and Over'
)
ORDER BY payments_per_enrollee DESC
"""

run_query(q3)

,demographic_group,total_enrollees,payments_per_enrollee,total_cost_sharing
0,Under 18 Years,1178,60014.0,8.631624e+06
1,95 Years and Over,458918,24139.0,1.268938e+09
2,85-94 Years,3228465,20754.0,9.656835e+09
3,55-64 Years,1737802,14292.0,4.233745e+09
4,75-84 Years,9754678,14133.0,2.327042e+10
5,45-54 Years,852996,12578.0,1.904409e+09
6,35-44 Years,569149,10969.0,1.128757e+09
7,25-34 Years,313894,9088.0,5.241220e+08
8,65-74 Years,17391872,8327.0,2.672691e+10
9,18-24 Years,58752,6645.0,7.449913e+07


Under 18 Years at $60,014 per enrollee is the highest. This makes sense as children on Medicare typically qualify due to disability or end-stage renal disease, making them extremely high-need patients.
95+ Years at $24,139 is second highest, expected given end-of-life care intensity.
Surprisingly, 65-74 Years at $8,327 is lower than 55-64 Years at $14,292. This is a well-known phenomenon called the "welcome to Medicare" effect where newly eligible 65-year-olds are generally healthier than the disabled under-65 population.

In [10]:
# Query 4: Payments per enrollee by race
q4 = """
SELECT 
    demographic_group,
    total_enrollees,
    ROUND(payments_per_enrollee, 0) AS payments_per_enrollee,
    ROUND(cost_sharing_per_enrollee, 0) AS cost_sharing_per_enrollee
FROM demographics
WHERE demographic_group IN (
    'Non-Hispanic White', 'Black (or African-American)',
    'Asian/Pacific Islander', 'Hispanic',
    'American Indian/Alaska Native'
)
ORDER BY payments_per_enrollee DESC
"""

run_query(q4)

,demographic_group,total_enrollees,payments_per_enrollee,cost_sharing_per_enrollee
0,American Indian/Alaska Native,158825,14896.0,2584.0
1,Black (or African-American),2714198,13808.0,2171.0
2,Non-Hispanic White,26467086,11982.0,2068.0
3,Hispanic,2444353,10632.0,1612.0
4,Asian/Pacific Islander,1244924,8749.0,1363.0


American Indian/Alaska Native beneficiaries have the highest spend at $14,896 — reflecting higher rates of chronic conditions like diabetes in this population.
Black beneficiaries at $13,808 are significantly higher than Non-Hispanic White at $11,982 — a $1,826 gap per enrollee that reflects well-documented healthcare disparities.
Asian/Pacific Islander at $8,749 is the lowest — nearly 70 percent less than American Indian/Alaska Native, which is a striking disparity

In [12]:
# Query 5: Total Medicare spend by service type over time
q5 = """
SELECT 
    service_type,
    year,
    ROUND(program_payments / 1000000000, 2) AS payments_billions
FROM trends
WHERE service_type NOT IN ('Total Part A and B', 'Part A Total', 'Part B Total')
ORDER BY service_type, year
"""

run_query(q5)

,service_type,year,payments_billions
0,Home Health Agency (Part A),2018,6.80
1,Home Health Agency (Part A),2019,6.70
2,Home Health Agency (Part A),2020,6.29
3,Home Health Agency (Part A),2021,6.11
4,Home Health Agency (Part A),2022,5.91
5,Home Health Agency (Part A),2023,5.87
6,Home Health Agency (Part B),2018,11.13
7,Home Health Agency (Part B),2019,11.16
8,Home Health Agency (Part B),2020,10.79
9,Home Health Agency (Part B),2021,10.77


Inpatient Hospital dropped from $138.53B to $130.16B in 2020 -people avoided hospitals during COVID
Outpatient Services dropped from $81.02B to $74.81B in 2020 - same effect
Hospice actually increased through COVID - going from $19.25B in 2018 to $22.45B in 2020 and continuing up to $25.67B in 2023
Physician services bounced back strongly - from $106.55B in 2020 to $126.34B in 2023, the highest in the entire period

In [14]:
# Query 6: Medicaid dual enrollees vs non-dual enrollees
q6 = """
SELECT 
    demographic_group,
    total_enrollees,
    ROUND(payments_per_enrollee, 0) AS payments_per_enrollee,
    ROUND(cost_sharing_per_enrollee, 0) AS cost_sharing_per_enrollee
FROM demographics
WHERE demographic_group IN ('MME', 'Non-MME')
ORDER BY payments_per_enrollee DESC
"""

run_query(q6)

,demographic_group,total_enrollees,payments_per_enrollee,cost_sharing_per_enrollee
0,MME,4869432,21394.0,3369.0
1,Non-MME,29498271,10228.0,1776.0


Medicaid dual enrollees (MME) cost $21,394 per enrollee - more than double the $10,228 for non-dual enrollees. This is one of the most important cost drivers in all of Medicare.
The cost sharing difference is also notable, $3,369 vs $1,776, meaning they're paying more out of pocket despite being the most financially vulnerable population.

Additional Analysis

In [19]:
# Query 8 fixed: Beneficiary cost burden ratio by state
q8 = """
SELECT 
    state,
    ROUND(payments_per_enrollee, 0) AS payments_per_enrollee,
    ROUND(beneficiary_cost_sharing * 1.0 / total_enrollees, 0) AS cost_sharing_per_enrollee,
    ROUND(beneficiary_cost_sharing * 100.0 / total_program_payments, 1) AS cost_burden_pct
FROM states
ORDER BY cost_burden_pct DESC
LIMIT 15
"""

run_query(q8)

,State,payments_per_enrollee,cost_sharing_per_enrollee,cost_burden_pct
0,Iowa,10513.0,2237.0,21.3
1,Montana,9170.0,1922.0,21.0
2,South Dakota,12464.0,2565.0,20.6
3,Wyoming,10341.0,2116.0,20.5
4,Nebraska,11689.0,2360.0,20.2
5,North Dakota,12125.0,2392.0,19.7
6,Idaho,9761.0,1908.0,19.6
7,Kansas,11632.0,2234.0,19.2
8,Vermont,10398.0,1984.0,19.1
9,Wisconsin,11044.0,2111.0,19.1


Iowa has the highest cost burden at 21.3% meaning patients there pay 21 cents out of every dollar spent on their care. And notably these are all lower-cost Midwestern and Western states like Iowa, Montana, South Dakota, Wyoming, Nebraska. The high-cost Northeastern states like New York and New Jersey don't appear here at all.
This tells a compelling health equity story where patients in lower-cost states actually bear a higher financial burden relative to what Medicare pays.

In [21]:
# Query 10: Racial disparity index vs overall average
q10 = """
WITH avg_payment AS (
    SELECT AVG(payments_per_enrollee) AS national_avg
    FROM demographics
    WHERE demographic_group = 'Total'
)
SELECT 
    d.demographic_group,
    ROUND(d.payments_per_enrollee, 0) AS payments_per_enrollee,
    ROUND(a.national_avg, 0) AS national_avg,
    ROUND((d.payments_per_enrollee - a.national_avg) / a.national_avg * 100, 1) AS disparity_pct
FROM demographics d, avg_payment a
WHERE d.demographic_group IN (
    'Non-Hispanic White', 'Black (or African-American)',
    'Asian/Pacific Islander', 'Hispanic',
    'American Indian/Alaska Native'
)
ORDER BY disparity_pct DESC
"""

run_query(q10)

,demographic_group,payments_per_enrollee,national_avg,disparity_pct
0,Non-Hispanic White,11982.0,None,None
1,Black (or African-American),13808.0,None,None
2,Asian/Pacific Islander,8749.0,None,None
3,Hispanic,10632.0,None,None
4,American Indian/Alaska Native,14896.0,None,None


In [23]:
# Query 10 fixed: Racial disparity index vs national average
q10 = """
WITH national_avg AS (
    SELECT ROUND(SUM(total_program_payments) * 1.0 / SUM(total_enrollees), 0) AS avg_per_enrollee
    FROM states
)
SELECT 
    d.demographic_group,
    ROUND(d.payments_per_enrollee, 0) AS payments_per_enrollee,
    ROUND(n.avg_per_enrollee, 0) AS national_avg,
    ROUND((d.payments_per_enrollee - n.avg_per_enrollee) * 100.0 / n.avg_per_enrollee, 1) AS disparity_pct
FROM demographics d, national_avg n
WHERE d.demographic_group IN (
    'Non-Hispanic White', 'Black (or African-American)',
    'Asian/Pacific Islander', 'Hispanic',
    'American Indian/Alaska Native'
)
ORDER BY disparity_pct DESC
"""

run_query(q10)

,demographic_group,payments_per_enrollee,national_avg,disparity_pct
0,American Indian/Alaska Native,14896.0,12023.0,23.9
1,Black (or African-American),13808.0,12023.0,14.8
2,Non-Hispanic White,11982.0,12023.0,-0.3
3,Hispanic,10632.0,12023.0,-11.6
4,Asian/Pacific Islander,8749.0,12023.0,-27.2


American Indian/Alaska Native beneficiaries cost 23.9% above the national average — the largest disparity in the dataset. Black beneficiaries are 14.8% above average while Asian/Pacific Islander beneficiaries are 27.2% below average — a combined spread of over 50 percentage points between the highest and lowest groups. Non-Hispanic White beneficiaries are essentially at the national average at -0.3%, which is also a meaningful finding.

In [25]:
# Export all query results for Tableau
run_query(q1).to_csv('output/tableau_top10_states.csv', index=False)
run_query(q2).to_csv('output/tableau_bottom10_states.csv', index=False)
run_query(q3).to_csv('output/tableau_age_groups.csv', index=False)
run_query(q4).to_csv('output/tableau_race.csv', index=False)
run_query(q5).to_csv('output/tableau_trends.csv', index=False)
run_query(q8).to_csv('output/tableau_cost_burden.csv', index=False)
run_query(q10).to_csv('output/tableau_disparity_index.csv', index=False)

# Also save the full states table for the map
run_query("SELECT * FROM states").to_csv('output/tableau_all_states.csv', index=False)

print("All files exported!")

All files exported!
